# Estimating capacity using boxing constraints

In [2]:
import pandas as pd

from cvx.simulator.builder import builder

In [5]:
# two assets, A and B, constant price for A=100 and B=200
prices=pd.read_csv("data/prices.csv", header=0, index_col=0, parse_dates=True)
prices

,A,B
date,,
2022-01-01,100.0,200.0
2022-01-02,100.0,200.0
2022-01-03,100.0,200.0
2022-01-04,100.0,200.0


In [9]:
# two assets, A and B, constant cap for A=10m and B=20m
market_cap=pd.read_csv("data/market_cap.csv", header=0, index_col=0, parse_dates=True)
market_cap

,A,B
date,,
2022-01-01,10000000.0,20000000.0
2022-01-02,10000000.0,20000000.0
2022-01-03,10000000.0,20000000.0
2022-01-04,10000000.0,20000000.0


In [10]:
# two assets, A and B, constant volume for A=1m and B=2m (volume measued in USD)
volume=pd.read_csv("data/volume.csv", header=0, index_col=0, parse_dates=True)
volume

,A,B
date,,
2022-01-01,1000000.0,2000000.0
2022-01-02,1000000.0,2000000.0
2022-01-03,1000000.0,2000000.0
2022-01-04,1000000.0,2000000.0


In [12]:
# some weights for A and B, position = weight * nav!
weights=pd.read_csv("data/target_weights.csv", header=0, index_col=0, parse_dates=True)
weights

,A,B
date,,
2022-01-01,0.25,0.75
2022-01-02,0.50,1.50
2022-01-03,0.75,2.30
2022-01-04,1.00,4.00


Given the very moderate size of the market both for A and B we may not
be able to enter a position as desired. We assume we have an allocated
capital of 1m. All numbers picked are unrealistic!



In [15]:
portfolio = builder(
    prices=prices,
    initial_cash=1e6,
    market_cap=market_cap,
    trade_volume=volume,
    weights=weights,
    max_cap_fraction=0.06,  # position max 6% of market cap
    max_trade_fraction=0.2, # trade max 20% of daily volume
).build()


In [16]:
portfolio.weights

,A,B
date,,
2022-01-01,0.2,0.4
2022-01-02,0.4,0.8
2022-01-03,0.6,1.2
2022-01-04,0.6,1.2


In [18]:
# a smaller portfolio won't collide with the box
portfolio = builder(
    prices=prices,
    initial_cash=1e5,
    market_cap=market_cap,
    trade_volume=volume,
    weights=weights,
    max_cap_fraction=0.06,  # position max 6% of market cap
    max_trade_fraction=0.2, # trade max 20% of daily volume
).build()

portfolio.weights

,A,B
date,,
2022-01-01,0.25,0.75
2022-01-02,0.50,1.50
2022-01-03,0.75,2.30
2022-01-04,1.00,4.00


In [19]:
# a larger portfolio will be far off the target position
portfolio = builder(
    prices=prices,
    initial_cash=1e7,
    market_cap=market_cap,
    trade_volume=volume,
    weights=weights,
    max_cap_fraction=0.06,  # position max 6% of market cap
    max_trade_fraction=0.2, # trade max 20% of daily volume
).build()

portfolio.weights

,A,B
date,,
2022-01-01,0.02,0.04
2022-01-02,0.04,0.08
2022-01-03,0.06,0.12
2022-01-04,0.06,0.12
